In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
# from numba import njit

In [2]:
a = np.linspace(1,100,100).reshape(10,10)
a

array([[  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.],
       [ 11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.],
       [ 21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.],
       [ 31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.],
       [ 41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.],
       [ 51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.],
       [ 61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.],
       [ 71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.],
       [ 81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.],
       [ 91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.]])

In [21]:
b = np.random.choice(len(a), 10, replace=False)
b

array([8, 7, 5, 1, 4, 3, 0, 6, 9, 2])

In [22]:
c = a[b]
c

array([[ 81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.],
       [ 71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.],
       [ 51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.],
       [ 11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.],
       [ 41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.],
       [ 31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.],
       [  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.],
       [ 61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.],
       [ 91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.],
       [ 21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.]])

In [16]:
d = np.random.poisson(a)
d

array([[  1,   3,   3,   3,  11,   4,   8,   7,  10,  14],
       [ 14,  13,  14,  14,  13,  15,  10,  22,  21,  24],
       [ 28,  26,  22,  23,  28,  23,  29,  22,  26,  29],
       [ 41,  37,  35,  35,  29,  44,  31,  35,  45,  44],
       [ 44,  49,  37,  45,  48,  45,  43,  46,  51,  65],
       [ 50,  62,  52,  51,  49,  62,  57,  67,  55,  53],
       [ 53,  72,  76,  66,  48,  74,  81,  61,  61,  68],
       [ 66,  88,  71,  76,  80,  87,  84,  64,  79,  65],
       [ 69,  91,  92,  71,  94, 110, 102,  79,  86,  87],
       [ 79,  76,  99,  92,  94,  83,  99, 101,  92,  96]])

In [17]:
f = np.random.random((10,5))
f

array([[0.58377037, 0.79476176, 0.22616142, 0.11541755, 0.29701644],
       [0.88025723, 0.67648924, 0.27369881, 0.27680499, 0.05452556],
       [0.31842291, 0.69640922, 0.28951423, 0.88631987, 0.18637271],
       [0.34382806, 0.10001533, 0.74275733, 0.14156922, 0.98710892],
       [0.12874408, 0.26224592, 0.02029065, 0.88345719, 0.26362999],
       [0.26681873, 0.5050008 , 0.73548884, 0.29914806, 0.03302784],
       [0.71760223, 0.60675005, 0.46654653, 0.99903923, 0.55437134],
       [0.47633483, 0.32645809, 0.1420819 , 0.32485559, 0.43614371],
       [0.04795551, 0.78863396, 0.73337696, 0.91435782, 0.07239421],
       [0.91270861, 0.17745988, 0.85371158, 0.59734406, 0.49069189]])

In [21]:
g = np.argsort(f, axis=0)
g

array([[8, 3, 4, 0, 5],
       [4, 9, 7, 3, 1],
       [5, 4, 0, 1, 8],
       [2, 7, 1, 5, 2],
       [3, 5, 2, 7, 4],
       [7, 6, 6, 9, 0],
       [0, 1, 8, 4, 7],
       [6, 2, 5, 2, 9],
       [1, 8, 3, 8, 6],
       [9, 0, 9, 6, 3]])

In [22]:
h = np.take_along_axis(f, g, axis=0)
h

array([[0.04795551, 0.10001533, 0.02029065, 0.11541755, 0.03302784],
       [0.12874408, 0.17745988, 0.1420819 , 0.14156922, 0.05452556],
       [0.26681873, 0.26224592, 0.22616142, 0.27680499, 0.07239421],
       [0.31842291, 0.32645809, 0.27369881, 0.29914806, 0.18637271],
       [0.34382806, 0.5050008 , 0.28951423, 0.32485559, 0.26362999],
       [0.47633483, 0.60675005, 0.46654653, 0.59734406, 0.29701644],
       [0.58377037, 0.67648924, 0.73337696, 0.88345719, 0.43614371],
       [0.71760223, 0.69640922, 0.73548884, 0.88631987, 0.49069189],
       [0.88025723, 0.78863396, 0.74275733, 0.91435782, 0.55437134],
       [0.91270861, 0.79476176, 0.85371158, 0.99903923, 0.98710892]])

In [23]:
a = np.linspace(1,10,10)
a

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [25]:
a[:,np.newaxis][:,np.newaxis]

array([[[ 1.]],

       [[ 2.]],

       [[ 3.]],

       [[ 4.]],

       [[ 5.]],

       [[ 6.]],

       [[ 7.]],

       [[ 8.]],

       [[ 9.]],

       [[10.]]])